In [7]:
# -*- coding: utf-8 -*-
"""
Build SW-style observables for PS3 from FRED + BLS.

Outputs:
  - usmodel_data_new.xlsx  (sheet 'Obs' with columns:
      date, dy, dc, dinve, dw, hours, pinfobs, robs)
Conventions:
  - Quarterly frequency
  - Logs are natural logs * 100
  - Growth observables are first differences of the logged per-capita levels
  - Indices normalized at 2017Q3 = 1 for CE16OV (employment) and CNP16OV (population)
  - EFFR is averaged monthly->quarterly, then divided by 4 to make a quarterly "rate"
"""

import os, json, requests
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from fredapi import Fred

# --------------------------
# 0) YOUR API KEYS (embedded)
# --------------------------
FRED_API_KEY = "f01b09fa5b1f987f649d603ee3bf4f88"  # you provided this
BLS_KEY = "17480a40f59e4e07a0678178edbe6e0a"  # you provided this

fred = Fred(api_key=FRED_API_KEY)


# -----------------------------------
# 1) Helper: monthly -> quarterly avg
# -----------------------------------
def to_quarterly_avg(s):
    s = s.copy()
    s.index = pd.to_datetime(s.index)
    # 使用季度末别名以避免 FutureWarning
    return s.resample("QE").mean()


def ln100(x):
    # safe log: coerce non-positive to NaN to avoid -inf
    x = pd.to_numeric(x, errors="coerce")
    x = x.where(x > 0)
    return 100.0 * np.log(x)


def normalize_at(series, period_str="2017Q3"):
    s = pd.to_numeric(series, errors="coerce").astype(float).copy()
    per = pd.Period(period_str, freq="Q")
    if per not in s.index:
        raise KeyError(
            f"Anchor period {period_str} not found in series index. Available from {s.index.min()} to {s.index.max()}"
        )
    anchor = s.loc[per]
    if pd.isna(anchor) or anchor == 0:
        raise ValueError(f"Anchor value for {period_str} is missing or zero: {anchor}")
    return s / anchor


# ----------------------------------
# 2) Pull FRED series (metadata refs)
# ----------------------------------
# GDPC1 (real GDP, chained 2017$, quarterly)         — FRED: GDPC1
# GDPDEF (GDP deflator, 2017=100, quarterly)         — FRED: GDPDEF
# FPI (Fixed Private Investment, nominal $ SAAR, q)  — FRED: FPI
# PCEC (Nominal personal consumption exp., SAAR, m)  — FRED: PCEC
# CE16OV (Employment level, persons 16+, monthly)    — FRED: CE16OV
# CNP16OV (Civilian population 16+, monthly)         — FRED: CNP16OV
# FEDFUNDS (Effective Fed Funds Rate, monthly)       — FRED: FEDFUNDS

series_q = ["GDPC1", "GDPDEF", "FPI"]
qdf = pd.DataFrame()
for s in series_q:
    try:
        ser = fred.get_series(s).to_frame(s)
        ser.index = pd.PeriodIndex(ser.index, freq="Q")
        qdf = pd.concat([qdf, ser], axis=1)
    except Exception as e:
        print(f"Warning: failed to fetch {s} from FRED: {e}")

series_m = ["PCEC", "CE16OV", "CNP16OV", "FEDFUNDS"]
mdf = {}
for s in series_m:
    try:
        ser = fred.get_series(s).to_frame(s)
        ser.index = pd.to_datetime(ser.index)
        mdf[s] = ser
    except Exception as e:
        print(f"Warning: failed to fetch {s} from FRED: {e}")
mqdf = pd.DataFrame()
for s in series_m:
    if s in mdf:
        try:
            mqdf = pd.concat([mqdf, to_quarterly_avg(mdf[s][s])], axis=1)
        except Exception as e:
            print(f"Warning: failed to process monthly series {s}: {e}")
    else:
        mqdf[s] = np.nan
mqdf.columns = series_m
mqdf.index = pd.PeriodIndex(mqdf.index, freq="Q") if len(mqdf.index) > 0 else mqdf.index

df = pd.concat([qdf, mqdf], axis=1).sort_index()

# --------------------------------------
# 3) Pull BLS PRS85006103 via BLS v2 API
#     (Hourly Compensation Index 2017=100, quarterly, SA)
#     We mimic your example: JSON body + headers
# --------------------------------------
headers = {"Content-type": "application/json"}
payload = {
    "seriesid": [
        "PRS85006103"
    ],  # Nonfarm Business Sector: Hourly Compensation (Index 2017=100, Q, SA)
    "startyear": "1947",
    "endyear": "2025",
    "registrationkey": BLS_KEY,
}
rows = []
try:
    resp = requests.post(
        "https://api.bls.gov/publicAPI/v2/timeseries/data/",
        data=json.dumps(payload),
        headers=headers,
        timeout=60,
    )
    resp.raise_for_status()
    js = resp.json()
    for series in js.get("Results", {}).get("series", []):
        sid = series.get("seriesID")
        for item in series.get("data", []):
            per = item.get("period", "")
            if per.startswith("Q"):
                year = int(item.get("year"))
                qtr = int(per[-2:]) if len(per) == 3 else int(per[-1])
                try:
                    val = float(item.get("value"))
                except Exception:
                    val = np.nan
                rows.append([pd.Period(f"{year}Q{qtr}", freq="Q"), val])
except Exception as e:
    print(f"Warning: BLS request failed: {e}. Continuing without BLS series.")

if len(rows) > 0:
    prs = (
        pd.DataFrame(rows, columns=["date", "PRS85006103"])
        .set_index("date")
        .sort_index()
    )
    df = df.join(prs, how="outer")
else:
    # ensure column exists for later code
    df["PRS85006103"] = np.nan

# --------------------------------------------
# 4) Trim the master panel to a safe broad span
#     (lets us compute lags before 1965Q1 window)
# --------------------------------------------
try:
    df = df[(df.index >= pd.Period("1955Q1")) & (df.index <= pd.Period("2025Q2"))]
except Exception as e:
    print(f"Warning trimming df index: {e}")

# 添加调试：检查 df 形状和 NaN
print(f"df shape after trim: {df.shape}")
print(f"df columns with NaN counts: {df.isna().sum()}")

# --------------------------------------------
# 5) Construct indices normalized at 2017Q3 = 1
#    (PS3/Read_me convention for population & employment)
# --------------------------------------------
try:
    LNSindex = normalize_at(df.get("CNP16OV"), "2017Q3")  # population index
except Exception as e:
    raise RuntimeError(f"Failed to construct LNSindex: {e}")
try:
    CE16index = normalize_at(df.get("CE16OV"), "2017Q3")  # employment index
except Exception as e:
    raise RuntimeError(f"Failed to construct CE16index: {e}")

# ----------------------------------------------------
# 6) Apply SW transforms (be careful with denominators)
#     - All logs are ln(*)*100
#     - Per capita uses CNP16OV index (LNSindex)
#     - Hours: (Hours index * Employment index / 100) / Pop index
#     - Inflation: ln(GDPDEF/GDPDEF(-1))*100
#     - Real wage: ln(PRS85006103 / GDPDEF)*100
#     - Policy rate (robs): quarterly 'level' = (avg monthly EFFR)/4
# ----------------------------------------------------
GDPDEF = pd.to_numeric(df.get("GDPDEF"), errors="coerce")
GDPC1 = pd.to_numeric(df.get("GDPC1"), errors="coerce")
PCEC = pd.to_numeric(df.get("PCEC"), errors="coerce")
FPI = pd.to_numeric(df.get("FPI"), errors="coerce")
# 确保 HRSIDX 为 Series（若列不存在则创建全 NaN 的 Series）
if "PRS85006023" in df.columns:
    HRSIDX = pd.to_numeric(df["PRS85006023"], errors="coerce")
else:
    HRSIDX = pd.Series(index=df.index, dtype=float)
HCIDX = pd.to_numeric(df.get("PRS85006103"), errors="coerce")
EFFR = pd.to_numeric(df.get("FEDFUNDS"), errors="coerce")

# If PRS85006023 isn't pre-fetched (rare), grab it quickly
if HRSIDX.isna().all():
    try:
        hrs = fred.get_series("PRS85006023").to_frame("PRS85006023")
        hrs.index = pd.PeriodIndex(hrs.index, freq="Q")
        df = df.join(hrs, how="left")
        HRSIDX = pd.to_numeric(df.get("PRS85006023"), errors="coerce")
        if not isinstance(HRSIDX, pd.Series):
            HRSIDX = pd.Series(HRSIDX, index=df.index)
    except Exception as e:
        print(f"Warning: failed to fetch PRS85006023: {e}")

# Real per-capita levels
consumption = ln100((PCEC / GDPDEF) / LNSindex)
investment = ln100((FPI / GDPDEF) / LNSindex)
output = ln100(GDPC1 / LNSindex)

# Hours per capita (aggregate hours proxy)
hours = ln100(((HRSIDX * CE16index) / 100.0) / LNSindex)

# Inflation (q/q)
pinfobs = ln100(GDPDEF / GDPDEF.shift(1))

# Real wage (hourly compensation / GDP deflator)
dw_level = ln100(HCIDX / GDPDEF)

# Policy rate (quarterly)
robs = EFFR / 4.0

# Growth observables (first differences)
dy = output.diff()
dc = consumption.diff()
dinve = investment.diff()
dw = dw_level.diff()

# 构建 obs，保留 NaN 值
obs = pd.DataFrame(
    {
        "dy": dy,
        "dc": dc,
        "dinve": dinve,
        "dw": dw,
        "hours": hours,
        "pinfobs": pinfobs,
        "robs": robs,
    }
)

# 添加调试：检查 obs 形状
print(f"obs shape after construction: {obs.shape}")
print(f"obs index range: {obs.index.min()} to {obs.index.max()}")

# Start the observable panel at 1965Q1 as per PS3
obs = obs[obs.index >= pd.Period("1965Q1")]

# --------------------------------------------
# 7) Write Excel similar to usmodel_data.xls
#     - Sheet 'Obs'
#     - Column A: date string (YYYYQn)
#     - Columns B:H: the 7 observables
# --------------------------------------------
sheet = obs.copy()
sheet.insert(0, "date", sheet.index.astype(str))

out_path = "usmodel_data_new.xlsx"
try:
    with pd.ExcelWriter(out_path, engine="openpyxl") as xlw:
        sheet.to_excel(xlw, sheet_name="Obs", index=False)
    print(f"Wrote {out_path} with sheet 'Obs' and columns:")
    print(list(sheet.columns))
    print(
        f"Rows: {len(sheet)} (from {sheet['date'].iloc[0]} to {sheet['date'].iloc[-1]})"
    )
except Exception as e:
    warn_path = "usmodel_data_new.csv"
    print(
        f"Warning: failed to write Excel file: {e}. Falling back to CSV at {warn_path}"
    )
    sheet.to_csv(warn_path, index=False)
    print(f"Wrote fallback CSV {warn_path} with columns: {list(sheet.columns)}")

df shape after trim: (282, 8)
df columns with NaN counts: GDPC1            0
GDPDEF           0
FPI              0
PCEC             0
CE16OV           0
CNP16OV          0
FEDFUNDS         0
PRS85006103    234
dtype: int64
obs shape after construction: (282, 7)
obs index range: 1955Q1 to 2025Q2
Wrote usmodel_data_new.xlsx with sheet 'Obs' and columns:
['date', 'dy', 'dc', 'dinve', 'dw', 'hours', 'pinfobs', 'robs']
Rows: 242 (from 1965Q1 to 2025Q2)
obs shape after construction: (282, 7)
obs index range: 1955Q1 to 2025Q2
Wrote usmodel_data_new.xlsx with sheet 'Obs' and columns:
['date', 'dy', 'dc', 'dinve', 'dw', 'hours', 'pinfobs', 'robs']
Rows: 242 (from 1965Q1 to 2025Q2)


In [ ]:
# -*- coding: utf-8 -*-
"""
PS3 data builder: fetch FRED+BLS, quarterly-average required monthly series,
store ALL fetched/constructed series in sheet 'data',
then compute SW-style observables and store them in sheet 'Obs'.

Output file:  SW Data4HW.xlsx
Sheets:
  - data: quarterly panel 1955Q1..2025Q2 (original/averaged series + indices + transformed levels)
  - Obs : dy, dc, dinve, dw, hours, pinfobs, robs; dates in col A; data start at B2; 1965Q1..2025Q2

References:
- BLS API v2 (≤ 20 years per call; JSON body with registrationkey)
- FRED series: GDPC1 (Q), GDPDEF (Q), FPI (Q), PCEC (M->Q avg), CE16OV (M->Q avg),
               CNP16OV (M->Q avg), FEDFUNDS (M->Q avg), PRS85006023 (Q), PRS85006103 (Q via BLS)
"""

import os, json, requests
import pandas as pd
import numpy as np
from fredapi import Fred

# ========= YOUR KEYS =========
FRED_API_KEY = "f01b09fa5b1f987f649d603ee3bf4f88"
BLS_KEY      = "17480a40f59e4e07a0678178edbe6e0a"
# =============================

fred = Fred(api_key=FRED_API_KEY)

# ---------- helpers ----------
def to_quarterly_avg(s: pd.Series) -> pd.Series:
    """Monthly series -> quarterly average (calendar Q)"""
    s = s.copy()
    s.index = pd.to_datetime(s.index)
    # Q-DEC quarters, Pandas default
    return s.resample("QE").mean()

def ln100(x):
    x = pd.to_numeric(x, errors="coerce")
    x = x.where(x > 0)
    return 100.0 * np.log(x)

def normalize_at(series: pd.Series, period_str="2017Q3") -> pd.Series:
    s = pd.to_numeric(series, errors="coerce").astype(float).copy()
    anchor = s.loc[pd.Period(period_str, freq="Q")]
    if pd.isna(anchor) or anchor == 0:
        raise ValueError(f"Anchor {period_str} missing/zero for {series.name}")
    return s / anchor

def fetch_bls_quarterly_series_chunked(series_id: str,
                                       start_year: int = 1947,
                                       end_year:   int = 2025,
                                       chunk:      int = 20,
                                       bls_key:    str = None) -> pd.Series:
    """BLS v2 allows up to 20y per call; loop chunks and merge quarterly obs."""
    headers = {"Content-type": "application/json"}
    rows = []
    sy = start_year
    while sy <= end_year:
        ey = min(sy + chunk - 1, end_year)
        payload = {"seriesid": [series_id], "startyear": str(sy), "endyear": str(ey)}
        if bls_key:
            payload["registrationkey"] = bls_key
        r = requests.post("https://api.bls.gov/publicAPI/v2/timeseries/data/",
                          data=json.dumps(payload), headers=headers, timeout=60)
        r.raise_for_status()
        js = r.json()
        for series in js.get("Results", {}).get("series", []):
            for item in series.get("data", []):
                per = item.get("period", "")
                if per.startswith("Q"):
                    y = int(item["year"])
                    q = int(per[-2:]) if len(per) == 3 else int(per[-1])
                    val = float(item["value"])
                    rows.append([pd.Period(f"{y}Q{q}", freq="Q"), val])
        sy = ey + 1

    if not rows:
        return pd.Series(dtype=float)

    dfb = (pd.DataFrame(rows, columns=["date", series_id])
             .drop_duplicates(subset=["date"])
             .set_index("date")
             .sort_index())
    return dfb[series_id]

# ---------- 1) FRED pulls ----------
# Quarterly directly
q_names = ["GDPC1", "GDPDEF", "FPI"]
qdf = []
for s in q_names:
    ser = fred.get_series(s).to_frame(s)
    ser.index = pd.PeriodIndex(ser.index, freq="Q")
    qdf.append(ser)
qdf = pd.concat(qdf, axis=1)

# Monthly -> quarterly average
m_names = ["PCEC", "CE16OV", "CNP16OV", "FEDFUNDS"]
m_q_df = []
for s in m_names:
    m = fred.get_series(s).to_frame(s)
    m_q = to_quarterly_avg(m[s]).to_frame(s)
    m_q_df.append(m_q)
mqdf = pd.concat(m_q_df, axis=1)
mqdf.index = pd.PeriodIndex(mqdf.index, freq="Q")

# Weekly hours index (quarterly, 2017=100)
hrs = fred.get_series("PRS85006023").to_frame("PRS85006023")
hrs.index = pd.PeriodIndex(hrs.index, freq="Q")

# merge FRED pieces
df = pd.concat([qdf, mqdf, hrs], axis=1).sort_index()

# ---------- 2) BLS PRS85006103 (hourly compensation, nominal index 2017=100) ----------
try:
    hcomp = fetch_bls_quarterly_series_chunked("PRS85006103", 1947, 2025, 20, BLS_KEY)
except Exception as e:
    print(f"[WARN] BLS chunked fetch failed: {e}")
    hcomp = pd.Series(dtype=float)

if hcomp.empty or hcomp.isna().all():
    # Fallback to FRED nominal hourly compensation (same concept)
    comp = fred.get_series("COMPNFB").to_frame("COMPNFB")
    comp.index = pd.PeriodIndex(comp.index, freq="Q")
    df = df.join(comp, how="outer")
    df["PRS85006103"] = df["COMPNFB"]
else:
    df = df.join(hcomp.to_frame("PRS85006103"), how="outer")

# ---------- 3) Trim master quarterly panel ----------
df = df[(df.index >= pd.Period("1955Q1")) & (df.index <= pd.Period("2025Q2"))]

# ---------- 4) Build indices / transforms (levels) ----------
LNSindex  = normalize_at(df["CNP16OV"], "2017Q3")   # population index
CE16index = normalize_at(df["CE16OV"],  "2017Q3")   # employment index

GDPDEF = pd.to_numeric(df["GDPDEF"], errors="coerce")
GDPC1  = pd.to_numeric(df["GDPC1"],  errors="coerce")
PCEC   = pd.to_numeric(df["PCEC"],   errors="coerce")
FPI    = pd.to_numeric(df["FPI"],    errors="coerce")
HRSIDX = pd.to_numeric(df["PRS85006023"], errors="coerce")
HCIDX  = pd.to_numeric(df["PRS85006103"], errors="coerce")
EFFR   = pd.to_numeric(df["FEDFUNDS"],    errors="coerce")

# SW-style level variables (ln*100)
consumption = ln100((PCEC / GDPDEF) / LNSindex)
investment  = ln100((FPI  / GDPDEF) / LNSindex)
output      = ln100( GDPC1          / LNSindex)
hours       = ln100(((HRSIDX * CE16index)/100.0) / LNSindex)
pinfobs     = ln100(GDPDEF / GDPDEF.shift(1))
real_wage   = ln100(HCIDX / GDPDEF)                     # level
interest_q  = EFFR / 4.0                                # quarterly level

# ---------- 5) Build 'data' sheet (quarterly panel with original + transformed) ----------
# Assemble a clean quarterly panel for archival & cross-checks
data_panel = pd.DataFrame({
    # Original/quarterly-averaged series
    "GDPC1": GDPC1, "GDPDEF": GDPDEF, "PCEC_qavg": PCEC, "FPI": FPI,
    "CE16OV_qavg": df["CE16OV"], "CNP16OV_qavg": df["CNP16OV"], "FEDFUNDS_qavg": EFFR,
    "PRS85006023": HRSIDX, "PRS85006103": HCIDX,
    # Indices normalized at 2017Q3
    "CE16OVindex_2017Q3=1": CE16index, "LNSindex_2017Q3=1": LNSindex,
    # Transformed level variables (as in read_me)
    "consumption": consumption, "investment": investment, "output": output,
    "labobs": hours, "inflation": pinfobs, "real_wage": real_wage, "interest_rate": interest_q,
})

# Optional: a numeric quarter label like 1947.1 for readability
yq_numeric = pd.Series([float(f"{p.year}.{p.quarter}") for p in data_panel.index],
                       index=data_panel.index, name="date_y.q")

# ---------- 6) Build 'Obs' sheet (differences / observables) ----------
dy    = output.diff()
dc    = consumption.diff()
dinve = investment.diff()
dw    = real_wage.diff()
robs  = interest_q

obs = pd.DataFrame({
    "dy": dy, "dc": dc, "dinve": dinve, "dw": dw,
    "labobs": hours, "pinfobs": pinfobs, "robs": robs
})
# Restrict Obs to 1965Q1..2025Q2
obs = obs[(obs.index >= pd.Period("1965Q1")) & (obs.index <= pd.Period("2025Q2"))]

# Hard check: no NaNs in Obs after 1965Q1
if obs.isna().any().any():
    raise RuntimeError(f"NaNs remain in Obs after 1965Q1:\n{obs.isna().sum()}")

# ---------- 7) Write workbook exactly as PS3 requires ----------
out_path = "SW_Data4HW.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as xlw:
    # Sheet 1: data  ——  ALL fetched/constructed quarterly series for audit
    # Put a 'date' string col (YYYYQn) and y.q numeric column in front to mimic your example style.
    data_out = data_panel.copy()
    data_out.insert(0, "date", data_out.index.astype(str))     # e.g., 1965Q1
    data_out.insert(1, "date_y.q", yq_numeric.values)          # e.g., 1965.1
    data_out.to_excel(xlw, sheet_name="data", index=False)

    # Sheet 2: Obs —— PS3 measurement data (A列日期, B2开始是数据)
    obs_out = obs.copy()
    obs_out.insert(0, "date", obs_out.index.astype(str))       # A列是日期
    obs_out.to_excel(xlw, sheet_name="Obs", index=False)

print(f"✓ Wrote '{out_path}' with sheets: ['data','Obs']")
print(f"  data rows: {len(data_panel)} (1955Q1..2025Q2)")
print(f"  Obs  rows: {len(obs)} ({obs.index.min()}..{obs.index.max()})")
print("  Obs header:", ["date","dy","dc","dinve","dw","hours","pinfobs","robs"])


✓ Wrote 'SW_Data4HW.xlsx' with sheets: ['data','Obs']
  data rows: 282 (1955Q1..2025Q2)
  Obs  rows: 242 (1965Q1..2025Q2)
  Obs header: ['date', 'dy', 'dc', 'dinve', 'dw', 'hours', 'pinfobs', 'robs']
